# Train and Test a network on the dstl data set
This notebook uses the Faster R-CNN network architecture for training a network on the dstl data set. During training some validation output is produced. 

The second part of the notebook deals with testing.

In [ ]:
import os
import sys
import pprint

import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)
        
add_path('/home/ubuntu/src/py-faster-rcnn-windowed_input/caffe-fast-rcnn/python')
add_path('/home/ubuntu/src/py-faster-rcnn-windowed_input/lib')
import caffe
from datasets.factory import get_imdb, list_imdbs
from fast_rcnn.test import test_net, plot_all_bboxes
from fast_rcnn.train import get_training_roidb, train_net
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
print "Loaded caffe version {:s} from {:s}.".format(caffe.__version__, caffe.__path__[0])

In [ ]:
# configure plotting
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['figure.figsize'] = (15, 15)

caffe.set_mode_gpu()

In [ ]:
classes = (0, 1, 4, 7, 8, 9)
arch = 'VGG16'
appendix = '' # This will codify which classes to train (if not all, in which case this string should be empty)
infix = '.' # for directories
if classes is not None:
    appendix = '_'+''.join(['{:d}'.format(c) for c in classes])
    infix = '{:d}'.format(len(classes))+'_classes'

print infix, appendix

In [ ]:
this_dir = os.getcwd()
data_dir = os.path.join(this_dir, '..', 'data', 'voc')
model_dir = os.path.join(this_dir, '..', 'models', arch, infix)

## Draw network
We load some modules required only for drawing the network and then draw the network too.

In [ ]:
#this block defines a function to visualize network from prototxt model
#install pydot via: sudo apt-get install python-pydot
from caffe import draw
from caffe.proto import caffe_pb2
from google.protobuf import text_format
from IPython.display import Image

import tempfile as tp

def draw_network(model_file, rankdir = 'LR', draw_to_file = False):
    net = caffe_pb2.NetParameter()
    text_format.Merge(open(model_file).read(), net)
    if draw_to_file:
        tmp_file_name =  tp.mktemp(dir= os.getcwd()) + ".png"
        draw.draw_net_to_file(caffe_net=net, filename=tmp_file_name)
    return draw.draw_net(caffe_net=net,rankdir=rankdir)

In [ ]:
train_file = os.path.join(model_dir, 'train.prototxt')
test_file = os.path.join(model_dir, 'test.prototxt')
# Drawing the network doesn't seem to work with 'include' fields in a layer
network_im = draw_network(train_file, draw_to_file= False)
display(Image(network_im))

solver_file = os.path.join(model_dir, 'solver_unified.prototxt')
print solver_file

Fucntion copied from `train_net` that returns two databases containing the images and the regions of interest respectively 

In [ ]:
def combined_roidb(imdb_names):
    def get_roidb(imdb_name):
        print imdb_name
        imdb = get_imdb(imdb_name)
        print 'Loaded dataset `{:s}` for training'.format(imdb.name)
        imdb.set_proposal_method(cfg.TRAIN.PROPOSAL_METHOD)
        print 'Set proposal method: {:s}'.format(cfg.TRAIN.PROPOSAL_METHOD)
        roidb = get_training_roidb(imdb)
        return roidb

    roidbs = [get_roidb(s) for s in imdb_names.split('+')]
    roidb = roidbs[0]
    if len(roidbs) > 1:
        for r in roidbs[1:]:
            roidb.extend(r)
        imdb = datasets.imdb.imdb(imdb_names)
    else:
        imdb = get_imdb(imdb_names)
    return imdb, roidb

## Training

In [ ]:
cfg_from_file('../experiments/cfgs/faster_rcnn_end2end_dstl.yml')
cfg.TRAIN.SNAPSHOT_INFIX = 'third'
cfg.TRAIN.SNAPSHOT_ITERS = 100
cfg.PIXEL_MEANS = np.array([[[102.34775165, 93.19367343, 84.36433397]]])
print('Using config:')
pprint.pprint(cfg)
np.random.seed(cfg.RNG_SEED)
caffe.set_random_seed(cfg.RNG_SEED)
imdb_train, roidb_train = combined_roidb('dstl'+appendix+'_train')
imdb_val, roidb_val = combined_roidb('dstl'+appendix+'_val')
output_dir = '/data/output/faster_rcnn_end2end/dstl'+appendix+'_train'
print output_dir

In [ ]:
# pretrained_caffemodel = '/home/ubuntu/dstl/models/{:s}/{:s}/trained/adapted_from_{:s}_faster_rcnn_final.caffemodel'.format(arch, infix, arch)
pretrained_caffemodel = '/data/output/faster_rcnn_end2end/dstl{:s}_train/{:s}_faster_rcnn_second_iter_1000.caffemodel'.format(appendix, arch.lower())
# pretrained_caffemodel = None
if False:
    model_paths, history = train_net(
              solver_file, roidb_train, output_dir, roidb_val=roidb_val,                                                                                    
              pretrained_model=pretrained_caffemodel,                                                                                
              max_iters=10*cfg.TRAIN.SNAPSHOT_ITERS)     

## Testing

In [ ]:
trained_caffemodel = '/data/output/faster_rcnn_end2end/dstl{:s}_train/{:s}_faster_rcnn_third_iter_1000.caffemodel'.format(appendix, arch.lower())
# trained_caffemodel = pretrained_caffemodel
net = caffe.Net(test_file, trained_caffemodel, caffe.TEST)
net.name = os.path.splitext(os.path.basename(trained_caffemodel))[0]
print "Loaded network from {:s}".format(test_file)
print "loaded caffemodel from {:s}".format(trained_caffemodel)

In [ ]:
imdb = get_imdb('dstl{:s}_test'.format(appendix))

In [ ]:
test_net(net, imdb, max_per_image=1000, max_im=None, thresh=0.5, vis=True, crop_size=None)

In [ ]:
if False:
    caffemodels = ['/data/output/faster_rcnn_end2end/dstl{:s}_train/{:s}_faster_rcnn_second_iter_{:d}.caffemodel'.format(appendix, arch.lower(), i) for i in range(100, 1100, 100)]
    for caffemodel in caffemodels:
        net = caffe.Net(test_file, caffemodel, caffe.TEST)
        net.name = os.path.splitext(os.path.basename(caffemodel))[0]
        print net.name
        test_net(net, imdb, max_per_image=100, max_im=2, thresh=0.5, vis=True, crop_size=600)